In [1]:
# import all necessary packages
import tensorflow as tf
import os
import numpy as np
import glob
import time

In [16]:
# setup the constants
IMG_SHAPE = 224  # size of our input image needed for our model IMG_SHAPE x IMG_SHAPE x 3 (color)
MODEL_DIRECTORY = 'D:\CSUEB\lynne\model_checkpoint_512256_1000'
DATA_DIRECTORY = 'D:\CSUEB\lynne\DataSet_NPY'

# labels for the 2 classes
LABELS = ['Falling','Walking']

RESULT_DIFF_THRESHOLD = 0.1
SEQUENCE_LENGTH = 40
FEATURE_LENGTH = 1280

In [3]:
# function to load a model from an h5 file
def loadModelFrom_H5_File(model_file):
    new_model = tf.keras.models.load_model(model_file)
    
    return new_model

In [35]:
# function to load and prepare the video feature data from a file
def loadData(data_path, sequence_length, feature_length):
    features = np.load(data_path)
    padded_sequence = np.zeros((sequence_length, feature_length))
    padded_sequence[0:len(features)] = np.array(features)
    
    return np.expand_dims(padded_sequence,axis=0)

In [5]:
# function to use an LSTM model to make a prediction on live video data
def predict(input, model):
    prediction = model.predict(input, batch_size=1, verbose=0)
    
    return prediction

In [48]:
# function to make a classification decision and return a label
def makeDecision(predictions, class_labels):
    if predictions[0][0] < predictions[0][1] and (predictions[0][1] - predictions[0][0]) > RESULT_DIFF_THRESHOLD:
        label = class_labels[0]
    elif predictions[0][0] - predictions[0][1] > RESULT_DIFF_THRESHOLD :
        label = class_labels[1]
    else:
        label = "Unknown"
    return label

In [11]:
# load the saved LSTM model
model_files = os.path.join(MODEL_DIRECTORY, '*.hdf5')
print(model_files)
model_paths = tf.io.gfile.glob(model_files)
model_file = model_paths[0]
model = loadModelFrom_H5_File(model_file)   

D:\CSUEB\lynne\model_checkpoint_512256_1000\*.hdf5


In [37]:
# load the saved LSTM model
model = loadModelFrom_H5_File(os.path.join(MODEL_DIRECTORY, 'weights.1000-0.00.hdf5'))  

In [38]:
# get all of the filenames and paths for the testing dataset from the
# .txt file that was created from the FeatureExtractionNotebook
testing_file = os.path.join(DATA_DIRECTORY,'testing.txt')

with open(testing_file) as f:
    testing_list = [row.strip() for row in list(f)]

In [39]:
# make a folder to store all of the batch prediction results if it doesn't already exist
results_directory = os.path.join(DATA_DIRECTORY,'BatchTestingPredictionResults')
if not os.path.exists(results_directory):
    os.mkdir(results_directory)

In [55]:
# create a new .txt file to save the results of the batch prediction
timestr = time.strftime("%Y%m%d-%H%M%S")
batch_prediction_filename = "BatchPredictions_" + timestr + ".txt"
batch_prediction_filename = os.path.join(results_directory,batch_prediction_filename)
prediction_file = open(batch_prediction_filename, 'w+')

In [56]:
# make and save predictions for all of the testing files in testing_list
unexpected = 0
for i in range(len(testing_list)):
    features = loadData(testing_list[i], SEQUENCE_LENGTH, FEATURE_LENGTH)
    prediction = predict(features, model)
    classification = makeDecision(prediction, LABELS)
    prediction_file.write("Prediction: " + classification + "\n" +
                          "Prediction array: " + str(prediction) + "\n" +
                          "Prediction Data Source: " + testing_list[i] + "\n\n\n")
    #write out the prediction vector
    expected = os.path.basename(os.path.dirname(testing_list[i]))
    if str(classification) != str(expected):
        unexpected += 1
prediction_file.close()
print("Total: ", len(testing_list), " unexpected: ", unexpected)

Total:  122  unexpected:  0
